In [1]:
# Importing the required modules
# TODO: Check which libraries remain relevant at the end.
import numpy as np

import pandas as pd


import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objs as go

import os
from pathlib import Path


pwd = os.getcwd()


# Get the current working directory
cwd = os.getcwd()
# parent_cwd = cwd.parent.absolute()

par_par_cwd = Path(cwd).parent.parent




# Define own working directory here 
# os.chdir(r"/Users/philipp/econ/experiment_model_persuasion/")


pwd = os.getcwd()


# Get the current working directory
cwd = os.getcwd()
# parent_cwd = cwd.parent.absolute()

par_par_cwd = Path(cwd).parent.parent

par_par_cwd / "img"



# getting the name of the directory
# where the this file is present.
# current = os.path.dirname(os.path.realpath(__file__))
  
# Getting the parent directory name
# where the current directory is present.
parent = os.path.dirname(pwd)
grandparent = os.path.dirname(parent)
  
# adding the parent directory to 
# the sys.path.
sys.path.append(grandparent)



from functions_data_gen import *
from functions_plotting import *




In [2]:
# generate data five times

data_1 = generate_data(model_switching_point=40, model_change_sign="neg_to_pos", trend_abs=1, var_errors=10, obs=100, seed=12345)
data_2 = generate_data(model_switching_point=20, model_change_sign="pos_to_neg", trend_abs=1, var_errors=10, obs=100, seed=23456)
data_3 = generate_data(model_switching_point=60, model_change_sign="neg_to_pos", trend_abs=1, var_errors=10, obs=100, seed=34567)
data_4 = generate_data(model_switching_point=30, model_change_sign="pos_to_neg", trend_abs=5, var_errors=20, obs=100, seed=45678)
data_5 = generate_data(model_switching_point=50, model_change_sign="neg_to_pos", trend_abs=1, var_errors=30, obs=100, seed=56789)




In [5]:
# Plot truncated model graphs.
dict = {"1": data_1, "2": data_2, "3": data_3, "4": data_4, "5": data_5}


trunc = 80
# TODO: This changed here; did I replicate correctedly.
t_list_model = np.arange(0,trunc,1,dtype=int)

for i in ["1", "2", "3", "4", "5"]:
    gen_model_graphs_trunc(data= dict[i], model_space=t_list_model,trunc=trunc, target_directory= par_par_cwd / "otree_model_persuasion" / "_static" / "persuasion", data_name=i)

In [6]:
# plot truncated neutral graphs.
dict = {"1": data_1, "2": data_2, "3": data_3, "4": data_4, "5": data_5}

for i in ["1", "2", "3", "4", "5"]:
    gen_neutral_graphs_trunc(data= dict[i], trunc=80, target_directory= par_par_cwd / "otree_model_persuasion" / "_static" / "persuasion", data_name=i)
